# Importar librerias

In [1]:
import pandas as pd
import numpy as np
import emoji
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import nltk
from nltk.corpus import stopwords

import spacy
from spacy.lang.es.stop_words import STOP_WORDS

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer, AutoModel


c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Cargar datos de entrenamiento

In [2]:
path = '../notebooks/data/sentiment_analysis_dataset.csv'
data = pd.read_csv(path)
data.head()

,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed,scared
1,@shpiderduck,me siento abrumado❤,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed,scared
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed,scared
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed,scared
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed,scared


# Preprocesamiento (Limpieza, Tokenización, Stopwords y Lematización)

In [3]:
# Traducción de las etiquetas a español
translation = {
    'joyful': 'Alegre',
    'daring': 'Osado',
    'optimistic': 'Optimista',
    'playful': 'Jugueton',
    'powerful': 'Poderoso',
    'surprised': 'Sorprendido',
    'successful': 'Exitoso',
    'confident': 'Confiado',
    'peaceful': 'Tranquilo',
    'secure': 'Seguro',
    'thankful': 'Agradecido',
    'loving': 'Amoroso',
    'relaxed': 'Relajado',
    'responsive': 'Sensible',
    'sad': 'Triste',
    'sleepy': 'Adormilado',
    'isolated': 'Aislado',
    'stupid': 'Estupido',
    'mad': 'Histerico',
    'distant': 'Distante',
    'frustrated': 'Frustrado',
    'irritated': 'Irritado',
    'jealous': 'Celoso',
    'scared': 'Asustado',
    'embarrassed': 'Avergonzado',
    'overwhelmed': 'Agobiado',
}

In [4]:
data[['emotion', 'sentiment']] = data[['emotion', 'sentiment']].replace(to_replace = translation)
data.head()

,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",Agobiado,Asustado
1,@shpiderduck,me siento abrumado❤,"Jan 6, 2024 · 2:35 AM UTC",Agobiado,Asustado
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",Agobiado,Asustado
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",Agobiado,Asustado
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",Agobiado,Asustado


In [5]:
# target_map ={
#     'Alegre': 1, # Sentimiento 1
#     'Osado': 1,
#     'Optimista': 2,
#     'Jugueton': 3,
#     'Poderoso': 2, # Sentimiento 2
#     'Sorprendido': 4,
#     'Exitoso': 5, 
#     'Confiado': 6, 
#     'Tranquilo': 3, # Sentimiento 3
#     'Seguro': 7,
#     'Agradecido': 8,
#     'Amoroso': 9,
#     'Relajado': 10,
#     'Sensible': 11,
#     'Triste': 4, # Sentimiento 4
#     'Adormilado': 12,
#     'Aislado': 13,
#     'Estupido': 14,
#     'Histerico': 5, # Sentimiento 5
#     'Distante': 15,
#     'Frustrado': 16,
#     'Irritado': 17,
#     'Celoso': 18,
#     'Asustado': 6, # Sentimiento 6
#     'Avergonzado': 19,
#     'Agobiado': 20,
# }

In [6]:
# emotions = ['Osado', 'Optimista', 'Jugueton', 'Sorprendido', 'Exitoso', 'Confiado', 'Seguro', 'Agradecido', 'Amoroso', 'Relajado', 'Sensible', 
#            'Adormilado', 'Aislado', 'Histerico', 'Distante', 'Frustrado', 'Irritado', 'Celoso', 'Avergonzado', 'Agobiado']

In [7]:
# reversed_map = {value: sentiment for sentiment, value in target_map.items()}

In [8]:
sentiments_map = {
    'Osado': 'Felicidad',
    'Optimista': 'Felicidad',
    'Jugueton': 'Felicidad',
    'Sorprendido': 'Empoderado',
    'Exitoso': 'Empoderado',
    'Confiado': 'Empoderado',
    'Seguro': 'Paz',
    'Agradecido': 'Paz',
    'Amoroso': 'Paz',
    'Relajado': 'Paz',
    'Sensible': 'Paz',
    'Adormilado': 'Tristeza',
    'Aislado': 'Tristeza',
    'Histerico': 'Tristeza',
    'Distante': 'Furia',
    'Frustrado': 'Furia',
    'Irritado': 'Furia',
    'Celoso': 'Furia',
    'Avergonzado': 'Miedo',
    'Agobiado': 'Miedo',
}

In [9]:
# data['emotion'] = data['emotion'].map(target_map)
# data.head()

In [10]:
df = data[['text', 'emotion']].copy()

In [11]:
nltk.download('stopwords')
nltk.download('punkt')

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Definir stopwords en español
stop_words = set(stopwords.words('spanish')) | STOP_WORDS

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WilmarAl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WilmarAl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def preprocess_text(text):
    text = emoji.replace_emoji(text, replace='')
    text = text.lower()
    text = text.replace('á','a').replace('é','e')
    text = text.replace('í','i').replace('ó','o')
    text = text.replace('ú','u').replace('$','')
    text = text.replace('—','').replace('-',' ')
    text = text.replace('%','').replace('&','')
    text = text.replace('\n',' ').replace('\t',' ')
    text = text.replace("'","").replace('"','')
    text = text.replace(',','').replace('.','')
    text = text.replace(';','').replace(':','')
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Eliminar menciones
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)  # Eliminar hashtags
    text = re.sub(r'http\S+|www\.\S+', '', text)  # Eliminar URLs
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ ]', '', text)  # Eliminar caracteres especiales y emojis

    #doc = nlp(text)
    
    # Eliminar stopwords y lematizar
    #palabras_procesadas = [token.lemma_ for token in doc if token.text.lower() not in stop_words and not token.is_punct]
    
    return text

def text_to_vector(tokens, model):
    # Obtener el vector promedio de todas las palabras en el texto
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)  # Promedio de los vectores
    else:
        return np.zeros(model.vector_size)  # Si no hay palabras conocidas, devolver un vector de ceros

def vectorize_text_column(df, text_column, model_name='dccuchile/bert-base-spanish-wwm-cased', max_length=128):
    """
    Vectoriza una columna de texto de un DataFrame usando un modelo preentrenado de Hugging Face.

    Parámetros:
    - df: DataFrame que contiene la columna de texto.
    - text_column: Nombre de la columna de texto a vectorizar.
    - model_name: Nombre del modelo preentrenado de Hugging Face (por defecto: BETO para español).
    - max_length: Longitud máxima de los textos (por defecto: 128).

    Retorna:
    - embeddings: Array de numpy con los embeddings de los textos.
    """
    # 1. Cargar tokenizer y modelo preentrenado
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # 2. Tokenizar los textos de la columna
    texts = df[text_column].tolist()
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)

    # 3. Extraer embeddings
    with torch.no_grad():  # Desactiva el cálculo de gradientes para mayor eficiencia
        outputs = model(**inputs)
    
    # 4. Obtener el embedding de la oración (promedio de los embeddings de los tokens)
    sentence_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

    return sentence_embeddings

In [13]:
df['text'] = df['text'].apply(preprocess_text)
df.head()

,text,emotion
0,termine bien abrumado despues de hoy,Agobiado
1,me siento abrumado,Agobiado
2,me siento un poco abrumado por la cantidad de ...,Agobiado
3,salvador la unica persona que no la ha abrumad...,Agobiado
4,denme un helado o algo que ando full abrumado,Agobiado


In [14]:
# Vectorizar la columna de texto
embeddings = vectorize_text_column(df, text_column='text')

# Almacenar los embeddings como una lista en una nueva columna
df['embeddings'] = embeddings.tolist()

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Vectorizar la columna de texto
# embeddings = vectorize_text_column(df, text_column='emotion')

In [16]:
df.head()

,text,emotion,embeddings
0,termine bien abrumado despues de hoy,Agobiado,"[0.07573962211608887, -0.3575446903705597, -0...."
1,me siento abrumado,Agobiado,"[-0.3122999370098114, -0.301361620426178, -0.0..."
2,me siento un poco abrumado por la cantidad de ...,Agobiado,"[-0.1711559295654297, -0.18406972289085388, -0..."
3,salvador la unica persona que no la ha abrumad...,Agobiado,"[-0.08535804599523544, 0.13760820031166077, 0...."
4,denme un helado o algo que ando full abrumado,Agobiado,"[-0.1372677981853485, -0.2031136155128479, -0...."


# Vectorización

In [17]:
X = np.stack(df['embeddings'].values).astype(np.float32)

In [18]:
label_encoder = LabelEncoder()
y = df['emotion']  # Etiquetas (emotion)
y = label_encoder.fit_transform(df['emotion'])  # Convierte las etiquetas a números
num_classes = len(label_encoder.classes_)  # Número de clases
# y_encoded = label_encoder.fit_transform(y)  # Convertir etiquetas a números
# y_categorical = to_categorical(y_encoded)   # Convertir a one-hot encoding

In [19]:
# Convertir los datos a tensores de PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  # Usar torch.long para índices de clase

# Train Test Split

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, y_tensor, test_size=0.15, random_state=42 )

# Entrenamiento y desempeño de los modelos

In [21]:
# Definir una red neuronal simple para clasificación multiclase
class DeepNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 160)
        self.dropout = nn.Dropout(0.3)  # Añade dropout en las capas
        self.fc2 = nn.Linear(160, 80)
        self.dropout = nn.Dropout(0.3)  # Añade dropout en las capas
        self.fc3 = nn.Linear(80, 40)
        self.dropout = nn.Dropout(0.3)  # Añade dropout en las capas
        self.fc4 = nn.Linear(40, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [22]:
class_weights = torch.tensor([3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.6, 3.6, 3.6, 3.7])  # Ajusta los pesos según el desbalance

In [23]:
# Crear DataLoader para manejar batches
batch_size = 32  # Ajusta el tamaño del batch
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [24]:
# Inicializar el modelo
input_size = X_train.shape[1]  # Dimensión del embedding
model = DeepNN(input_size, num_classes)

In [25]:
# # Inicializar el modelo
# batch_size = 64  # Ajusta el tamaño del batch
learning_rate = 0.01
# num_epochs = 20
# input_size = X_train.shape[1]  # Dimensión del embedding
# model = DeepNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss(weight=class_weights)  # Para clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [26]:
# Entrenar el modelo
num_epochs = 40  # Ajusta el número de épocas
for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo entrenamiento
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Imprimir la pérdida promedio por época
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/40, Loss: 3.0005
Epoch 2/40, Loss: 2.9641
Epoch 3/40, Loss: 2.8421
Epoch 4/40, Loss: 2.7426
Epoch 5/40, Loss: 2.6208
Epoch 6/40, Loss: 2.5440
Epoch 7/40, Loss: 2.5262
Epoch 8/40, Loss: 2.4313
Epoch 9/40, Loss: 2.3602
Epoch 10/40, Loss: 2.3010
Epoch 11/40, Loss: 2.2519
Epoch 12/40, Loss: 2.2619
Epoch 13/40, Loss: 2.1675
Epoch 14/40, Loss: 2.1342
Epoch 15/40, Loss: 2.0667
Epoch 16/40, Loss: 2.0390
Epoch 17/40, Loss: 1.9784
Epoch 18/40, Loss: 1.9782
Epoch 19/40, Loss: 1.9846
Epoch 20/40, Loss: 1.9123
Epoch 21/40, Loss: 1.8797
Epoch 22/40, Loss: 1.8173
Epoch 23/40, Loss: 1.7965
Epoch 24/40, Loss: 1.7524
Epoch 25/40, Loss: 1.7483
Epoch 26/40, Loss: 1.8236
Epoch 27/40, Loss: 1.7801
Epoch 28/40, Loss: 1.6713
Epoch 29/40, Loss: 1.6737
Epoch 30/40, Loss: 1.6505
Epoch 31/40, Loss: 1.6026
Epoch 32/40, Loss: 1.5953
Epoch 33/40, Loss: 1.6150
Epoch 34/40, Loss: 1.6278
Epoch 35/40, Loss: 1.5870
Epoch 36/40, Loss: 1.5178
Epoch 37/40, Loss: 1.6350
Epoch 38/40, Loss: 1.5555
Epoch 39/40, Loss: 1.

In [27]:
# # Entrenar el modelo
# for epoch in range(num_epochs):
#     model.train() 
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#     optimizer.zero_grad()
#     outputs = model(X_train)
#     loss = criterion(outputs, Y_train)
#     loss.backward()
#     optimizer.step()
#     print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [28]:
# Evaluar el modelo
model.eval()  # Poner el modelo en modo evaluación
correct = 0
total = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)  # Obtener la clase predicha
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 19.02%


In [29]:
# # Evaluar el modelo
# model.eval()  # Poner el modelo en modo evaluación
# with torch.no_grad():
#     outputs = model(X_test)
#     _, predicted = torch.max(outputs, 1)  # Obtener la clase predicha
#     accuracy = (predicted == Y_test).float().mean()
#     print(f'Accuracy: {accuracy.item() * 100:.2f}%')

# Pruebas

In [30]:
# # Texto de entrada
# user_text = 'La verdad extraño la persona que solías ser, que me recordaba que debo descansar sin sentirme culpable, que se molestó en conocer mi lenguaje corporal y me hizo sentir que quererme no era difícil. Te lo agradezco, pero te llevaste mi parte más vulnerable y ahora no sé qué hacer.'

# # 1. Preprocesar el texto de entrada
# user_tokens = preprocess_text(user_text)  # Tokenización
# user_vector = text_to_vector(user_tokens, word2vec_model)  # Convertir a vector

# # 2. Convertir el vector en un formato compatible con el modelo
# user_vector = np.array([user_vector])  # Añadir una dimensión extra (batch size = 1)

# # 3. Hacer la predicción con la red neuronal
# prediction = model_ann.predict(user_vector)  # Obtener las probabilidades de cada clase
# predicted_class_index = np.argmax(prediction, axis=1)  # Obtener la clase predicha

# # 4. Convertir el índice de la clase predicha a la etiqueta original
# predicted_class = label_encoder.inverse_transform(predicted_class_index)

# # 5. Mostrar el resultado
# print(f"Texto: {user_text}")
# print(f"Clase predicha: {predicted_class[0]}")
# print(f"Probabilidades por clase: {prediction}")